In [147]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [148]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q,globals())

In [149]:
df = pd.read_csv('../../data/data/H161.csv', parse_dates=['utc_time'])
# df.columns = ['utc_time','73_H160.PHASE','73_H160.STEP','73_H160.STATMENT','73_H160.NN(33)','GWI4801.PV','GTI8031.PV','GPI8030.PV']
df.columns = ['time','phase', 'step', 'statement', 'heat_time', 'weight', 'temp', 'pressure']

/tmp/ipykernel_8304/3789146688.py:1: DtypeWarning: Columns (4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/data/H161.csv', parse_dates=['utc_time'])


In [150]:


df["time"] = df["time"].dt.tz_localize(None)
df['date'] = df['time'].dt.date

In [151]:
df.sample(2)

,time,phase,step,statement,heat_time,weight,temp,pressure,date
378938,2020-12-11 08:46:27,DISCHRG,STRT,21.0,0.0,3688.4,22.0,764.7,2020-12-11
129441,2020-06-21 02:29:27,FEEDING,LP01,1.0,0.0,3255.4,60.4,751.5,2020-06-21


In [158]:
def load_query(path):
    with open(path) as file:
        return file.read()

In [159]:
query = load_query('daily_cycles.sql')
daily_cycles = pysqldf(query)

In [166]:
daily_cycles


,DATE,num_cycles
0,2020-03-23,1
1,2020-03-24,1
2,2020-03-25,1
3,2020-03-26,1
4,2020-03-27,1
...,...,...
703,2022-03-09,1
704,2022-03-10,1
705,2022-03-11,1
706,2022-03-12,1


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036791 entries, 0 to 1036790
Data columns (total 9 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   time       1036791 non-null  datetime64[ns]
 1   phase      1027623 non-null  object        
 2   step       1027627 non-null  object        
 3   statement  1031566 non-null  float64       
 4   heat_time  1036786 non-null  object        
 5   weight     1036784 non-null  object        
 6   temp       1036782 non-null  object        
 7   pressure   1036780 non-null  object        
 8   date       1036791 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 71.2+ MB
